In [ ]:
!nvidia-smi

Fri Dec 17 09:33:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import pandas as pd
df = pd.read_csv('AbstractDataset_trainSet_FULL.csv')
df.head()

,id,file_name,text,#label
0,1,2111.09381_originalAbstract.txt,"We present MEDCOD, a Medically- Accurate, Emot...",1
1,2,2111.09381_generatedAbstract.txt,"We present MEDCOD, a Medically- Accurate, Emot...",0
2,3,2111.09388_originalAbstract.txt,This work applies Minimum Bayes Risk (MBR) dec...,1
3,4,2111.09388_generatedAbstract.txt,This work applies Minimum Bayes Risk (MBR) dec...,0
4,5,2111.09412_originalAbstract.txt,"In this study, we present a meta-learning mode...",1


In [ ]:
!git clone -b master https://github.com/charles9n/bert-sklearn
!cd bert-sklearn; pip install .
import os
os.chdir("bert-sklearn")
print(os.listdir())

Cloning into 'bert-sklearn'...
remote: Enumerating objects: 259, done.
remote: Total 259 (delta 0), reused 0 (delta 0), pack-reused 259
Receiving objects: 100% (259/259), 516.15 KiB | 4.82 MiB/s, done.
Resolving deltas: 100% (131/131), done.
Processing /content/bert-sklearn
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 131 kB 5.2 MB/s 
     |████████████████████████████████| 8.4 MB 42.5 MB/s 
     |████████████████████████████████| 79 kB 8.7 MB/s 
     |████████████████████████████████| 138 kB 52.9 MB/s 
     |████████████████████████████████| 127 kB 51.3 MB/s 
  Created wheel for bert-sklearn: filename

In [ ]:
import torch
print('pytorch version:', torch.__version__)
print('GPU:',torch.cuda.get_device_name(0))

pytorch version: 1.10.0+cu111
GPU: Tesla K80


In [ ]:
import os
import math
import random
import csv
import sys

import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
import statistics as stats

from bert_sklearn import BertClassifier
from bert_sklearn import BertRegressor
from bert_sklearn import BertTokenClassifier
from bert_sklearn import load_model

def read_tsv(filename, quotechar=None):
    with open(filename, "r", encoding='utf-8') as f:
        return list(csv.reader(f, delimiter="\t", quotechar=quotechar))   

def flatten(l):
    return [item for sublist in l for item in sublist]

def read_CoNLL2003_format(filename, idx=3):
    """Read file in CoNLL-2003 shared task format"""
    
    # read file
    lines =  open(filename).read().strip()   
    
    # find sentence-like boundaries
    lines = lines.split("\n\n")  
    
     # split on newlines
    lines = [line.split("\n") for line in lines]
    
    # get tokens
    tokens = [[l.split()[0] for l in line] for line in lines]
    
    # get labels/tags
    labels = [[l.split()[idx] for l in line] for line in lines]
    
    #convert to df
    data= {'tokens': tokens, 'labels': labels}
    df=pd.DataFrame(data=data)
    
    return df

In [ ]:
X = df['text']
y = df['#label']
df.shape

(200, 4)

In [ ]:
messages = df.copy()
messages.reset_index(inplace = True)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 0)

In [ ]:
model = BertClassifier(max_seq_length=25, train_batch_size=64)
model
#will change train_batch_size to 64 (previous is 16) and maximum length to 25 (previous is 64) as per the Maronikolakis et al., 2021

Building sklearn text classifier...


BertClassifier(max_seq_length=25, train_batch_size=64)

In [ ]:
%%time
model = model.fit(X_train, y_train)

100%|██████████| 231508/231508 [00:00<00:00, 1670845.03B/s]


Loading bert-base-uncased model...


100%|██████████| 433/433 [00:00<00:00, 321553.41B/s]


Defaulting to linear classifier/regressor
Loading Pytorch checkpoint

train data size: 144, validation data size: 16



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


...

Validating:   0%|          | 0/2 [00:00<?, ?it/s]


Epoch 2, Train loss: 0.7014, Val loss: 0.6961, Val accy: 50.00%



Training  :   0%|          | 0/3 [00:00<?, ?it/s]

Validating:   0%|          | 0/2 [00:00<?, ?it/s]


Epoch 3, Train loss: 0.6982, Val loss: 0.6977, Val accy: 50.00%

CPU times: user 19.6 s, sys: 7.7 s, total: 27.2 s
Wall time: 39.5 s


In [ ]:
from tqdm import tqdm
# score model
accy = model.score(X_test, y_test)

# make class probability predictions
y_prob = model.predict_proba(X_test)
print("class prob estimates:\n", y_prob)

# make predictions
y_pred = model.predict(X_test)
print("Accuracy: %0.2f%%"%(metrics.accuracy_score(y_pred, y_test) * 100))

target_names = ['negative', 'positive']
print(classification_report(y_test, y_pred, target_names=target_names))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Testing:   0%|          | 0/5 [00:00<?, ?it/s]


Loss: 0.7016, Accuracy: 50.00%


Predicting:   0%|          | 0/5 [00:00<?, ?it/s]

class prob estimates:
 [[0.5196192  0.4803808 ]
 [0.5029531  0.49704686]
 [0.5137157  0.48628438]
 [0.5314992  0.46850082]
 [0.513068   0.486932  ]
 [0.5232057  0.4767943 ]
 [0.5142558  0.4857442 ]
 [0.53573835 0.46426162]
 [0.5323463  0.46765366]
 [0.56115574 0.43884432]
 [0.52722985 0.47277015]
 [0.51821166 0.48178834]
 [0.5048222  0.49517784]
 [0.51188326 0.48811674]
 [0.5357048  0.46429524]
 [0.5096534  0.4903466 ]
 [0.5118989  0.48810112]
 [0.52602917 0.4739708 ]
 [0.5173437  0.48265627]
 [0.51746035 0.48253965]
 [0.5232057  0.4767943 ]
 [0.51708496 0.482915  ]
 [0.51985466 0.48014528]
 [0.5199404  0.48005962]
 [0.52722985 0.47277015]
 [0.50849104 0.49150893]
 [0.52279615 0.47720388]
 [0.5074353  0.49256465]
 [0.50849104 0.49150893]
 [0.51985466 0.48014528]
 [0.51832086 0.48167914]
 [0.50725156 0.49274847]
 [0.54741484 0.45258513]
 [0.5163001  0.48369992]
 [0.51723754 0.48276243]
 [0.51618576 0.4838142 ]
 [0.5155618  0.4844382 ]
 [0.5142558  0.4857442 ]
 [0.5208647  0.47913527]
 [

Predicting:   0%|          | 0/5 [00:00<?, ?it/s]

Accuracy: 50.00%
              precision    recall  f1-score   support

    negative       0.50      1.00      0.67        20
    positive       0.00      0.00      0.00        20

    accuracy                           0.50        40
   macro avg       0.25      0.50      0.33        40
weighted avg       0.25      0.50      0.33        40



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#if you want to save the model and use it later
#save model to disk
savefile = 'test.bin'
model.save(savefile)

# load model from disk
new_model = load_model(savefile)

# predict with new model
accy = new_model.score(X_test, y_test)

Loading model from test.bin...
Defaulting to linear classifier/regressor
Building sklearn text classifier...


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Testing:   0%|          | 0/5 [00:00<?, ?it/s]


Loss: 0.7016, Accuracy: 50.00%
